<a href="https://colab.research.google.com/github/gaddour77/PFEFlask/blob/main/Copie_de_Cr%C3%A9ation_de_la_base_vectorielle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y poppler-utils
!pip install pdf2image pytesseract pillow
!apt-get install -y tesseract-ocr-fra

import pytesseract
from pdf2image import convert_from_path
import os
from google.colab import drive

# Monter Google Drive

def ocr_pdf_to_text(pdf_path, output_txt_path, lang="fra"):
    """
    Convertit un PDF en texte brut via OCR et sauvegarde le résultat.
    """
    pages = convert_from_path(pdf_path)

    text = ""
    for page in pages:
        text += pytesseract.image_to_string(page, lang=lang) + "\n"

    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write(text)

    print(f"✅ Texte OCR extrait depuis {pdf_path} et sauvegardé dans {output_txt_path}")
    print("=== Aperçu ===")
    print(text[:500])  # afficher un extrait du texte brut
    print("\n")

# Exemple avec 2 PDF
pdf1 = "/content/drive/MyDrive/chunks/assurance_merged.pdf"
txt1 = "/content/drive/MyDrive/chunks/assurance_text.txt"

pdf2 = "/content/drive/MyDrive/chunks/code.pdf"
txt2 = "/content/drive/MyDrive/chunks/code_commerce_text.txt"
drive.mount('/content/drive')
ocr_pdf_to_text(pdf1, txt1)
ocr_pdf_to_text(pdf2, txt2)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.10).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-fra is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Mounted at /content/drive
✅ Texte OCR extrait depuis /content/drive/MyDrive/chunks/assurance_merged.pdf et sauvegardé dans /content/drive/MyDrive/chunks/assurance_text.txt
=== Aperçu ===
CODE DES ASSURANCES

TITREI
LE CONTRAT D'ASSURANCE

CHAPITRE I
DISPOSITIONS GENERALES

Article premier.- Le contrat d'assurance est la convention par
laquelle une entreprise d'assurance ou assureur s'engage, en cas de
réalisation du risque ou au terme fixé au contrat, à fournir à une autre
personne appelée "assuré" une prestation pécuniaire en co

In [ ]:
# Installation des bibliothèques nécessaires
!pip install -q -U sentence-transformers
!pip install -q numpy
!pip install -q scikit-learn

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import os
from google.colab import drive
drive.mount('/content/drive')
# --- 1. Lecture des fichiers ---
file_names = ["/content/drive/MyDrive/chunks/assurance_text.txt", "/content/drive/MyDrive/chunks/code_commerce_text.txt"]
full_text = ""

try:
    for file_name in file_names:
        with open(file_name, 'r', encoding='utf-8') as f:
            full_text += f.read() + "\n\n"
    print("Fichiers lus avec succès.")
except FileNotFoundError as e:
    print(f"Erreur : Un des fichiers n'a pas été trouvé. Veuillez vous assurer que le chemin '{e.filename}' est correct et que Google Drive est monté.")
    exit()

# --- 2. Chargement du modèle local ---
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Modèle de transformation de phrases chargé avec succès.")

# --- 3. Segmentation du texte et création de la base de données ---
articles = full_text.split("Article")
articles = [("Article " + item).strip() for item in articles if item.strip()]

# Créez la base de données vectorielle
vector_db = []
for art in articles:
    embedding = model.encode(art)
    vector_db.append({
        'text': art,
        'embedding': embedding
    })

print(f"Base de données vectorielle créée avec {len(vector_db)} articles.")

# --- 4. Exemple de recherche de similarité ---
if vector_db:
    query = "Droits et obligations des entreprises d'assurance."
    print(f"\nRecherche pour la requête : '{query}'")

    query_embedding = model.encode(query)

    similarities = []
    for item in vector_db:
        similarity = cosine_similarity([query_embedding], [item['embedding']])[0][0]
        similarities.append((similarity, item['text']))

    similarities.sort(key=lambda x: x[0], reverse=True)

    if similarities:
        print("\n--- Résultat le plus pertinent ---")
        best_match_score = similarities[0][0]
        best_match_text = similarities[0][1]
        print(f"Score de similarité: {best_match_score:.4f}")
        print(f"Article pertinent:\n{best_match_text}")

Mounted at /content/drive
Fichiers lus avec succès.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Modèle de transformation de phrases chargé avec succès.
Base de données vectorielle créée avec 1525 articles.

Recherche pour la requête : 'Droits et obligations des entreprises d'assurance.'

--- Résultat le plus pertinent ---
Score de similarité: 0.6945
Article pertinent:
Article  82.- Les professions du secteur des assurances sont
soumises au contrôle du « Comité Général des Assurances » (). Le
contrôle vise à protéger les assurés, les bénéficiaires et toute autre
tierce partie intéressée à la bonne exécution des contrats d'assurances.

"Le contrôle porte notamment sur l'application de la
réglementation des assurances, le fonctionnement des entreprises
d'assurances et des entreprises de réassurances et l'emploi des fonds
liés à des opérations d'assurances et des opérations de réassurances".
(Abrogé et remplacé par l'article premier de la loi n° 2002-37 du
1° avril 2002).

(1) L'expression est remplacée par art. 2 de la loi n°2008-8 du 13 février 2008

39


In [ ]:
import pickle

# --- Sauvegarde de la base de données ---
# Assurez-vous que le dossier 'chunks' existe dans votre Drive
# Le fichier sera sauvegardé sous le nom 'vector_db.pkl'
output_path = '/content/drive/MyDrive/chunks/vector_db.pkl'

try:
    with open(output_path, 'wb') as f:
        pickle.dump(vector_db, f)
    print(f"\nBase de données vectorielle sauvegardée avec succès à : {output_path}")
except Exception as e:
    print(f"\nErreur lors de la sauvegarde : {e}")


Base de données vectorielle sauvegardée avec succès à : /content/drive/MyDrive/chunks/vector_db.pkl
